In [1]:
import os

gpus = [0]
os.environ['CUDA_VISIBLE_DEVICES']=','.join([str(i) for i in gpus])

In [2]:
import tensorflow as tf 
import tensorflow.keras as keras
import numpy as np 
#from oe_acute import MNE
import pickle
import sys
import random
#from AE import MDSAE as ae
#from network_visualisation import plot_these_aud_weights
#import network_visualisation
#import quantify_aud_strfs
import matplotlib.pyplot as plt

In [3]:
bird = 'B1692'
d = 32

### Allocate GPUs

In [4]:
gpus = [0] # Here I set CUDA to only see one GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=','.join([str(i) for i in gpus])
num_gpus = len(gpus) # number of GPUs to use
if len(gpus) < 1:
    num_gpus = 1 

In [5]:
from tensorflow.python.client import device_lib
local_device_protos = device_lib.list_local_devices()
print( [x.name for x in local_device_protos if x.device_type == 'GPU'] )

['/device:GPU:0']


### Model function

In [6]:
class Temporal_Specgram_CNN_Model(keras.Model):
    def __init__(self, **kwargs):
        super(Temporal_Specgram_CNN_Model, self).__init__()
        self.__dict__.update(kwargs)
        self.enc_1=keras.Sequential([
            keras.Input(shape=(32, 32, 1)),
#             keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation="relu",),
            keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1, 1), activation="relu",),
            tf.keras.layers.Flatten(),
            
            
        ])
        self.enc_dropout=tf.keras.layers.Dropout(0.5)
        self.enc_2=tf.keras.layers.Dense(units=d,activation='sigmoid', kernel_regularizer=keras.regularizers.L1(10**-3.5),)
        self.dec_recon=keras.Sequential([
            tf.keras.layers.Dense(units=16 * 16 * 256, activation="relu", kernel_regularizer=keras.regularizers.L1(10**-3.5)),
            tf.keras.layers.Reshape(target_shape=(16, 16, 256)),
#             tf.keras.layers.Conv2DTfranspose(
#                 filters=16, kernel_size=2, strides=(2, 2),  activation="relu", kernel_regularizer=keras.regularizers.L1(10**-3.5), 
#             ),
            tf.keras.layers.Conv2DTranspose(
                filters=8, kernel_size=2, strides=(2, 2), activation="relu", kernel_regularizer=keras.regularizers.L1(10**-3.5)
            ),
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=1, strides=(1, 1), kernel_regularizer=keras.regularizers.L1(10**-3.5)
            ),
        ])
        self.dec_pred=keras.Sequential([tf.keras.layers.Dense(units=32, kernel_regularizer=keras.regularizers.L1(10**-3.5))])
        self.recon_losses=[]
        self.pred_losses=[]
    @tf.function
    def get_loss(self, x_t, y_t):
        #print(x_t)#.shape
        x_hat, y_hat = self(tf.expand_dims(x_t, -1))
        pred_losses=tf.reduce_mean(tf.square(y_t - y_hat))
        
        recon_losses=tf.reduce_mean(tf.square(x_t - tf.squeeze(x_hat, -1)))

        
        return pred_losses,recon_losses

    @tf.function
    def get_gradients(self, x_t, y_t):
        with tf.GradientTape() as tape:
            pred_losses,recon_losses = self.get_loss(x_t, y_t)
            #tf.print(pred_losses, recon_losses)
            loss=pred_losses+0.5*recon_losses
            
        return loss, tape.gradient(loss, self.enc_1.trainable_variables+self.enc_2.trainable_variables+self.dec_recon.trainable_variables+self.dec_pred.trainable_variables)
    
    @tf.function
    def train_model(self, X_train, y_train):
        loss, gradients = self.get_gradients(X_train, y_train)
        self.optimizer.apply_gradients(zip(gradients, self.enc_1.trainable_variables+self.enc_2.trainable_variables+self.dec_recon.trainable_variables+self.dec_pred.trainable_variables))
        return loss

    @tf.function
    def compute_test_loss(self, X_test, y_test):
        x_hat, y_hat = self.predict(tf.expand_dims(X_test, -1))
        pred_losses=tf.reduce_mean(tf.square(y_test - y_hat))
        
        recon_losses=tf.reduce_mean(tf.square(X_test - tf.squeeze(x_hat, -1)))

        
        return pred_losses,recon_losses


    def call(self, input):
        latent=self.enc_1(input)
        latent=self.enc_dropout(latent, training=True)
        latent=self.enc_2(latent)
        return self.dec_recon(latent), self.dec_pred(latent)

    def predict(self, input):
        latent=self.enc_1(input)
        latent=self.enc_dropout(latent, training=False)
        latent=self.enc_2(latent)
        return self.dec_recon(latent), self.dec_pred(latent)


In [7]:
#model.enc.

In [8]:
#x_t, y_t = train_batch[0], train_batch[1]

In [9]:
#pred_losses,recon_losses=model.get_loss(x_t, y_t)

In [10]:
#pred_losses

### Load data

In [11]:
#def extract_spec_data(x_path, y_path, random_seed=None, global_normalize=False):
def extract_spec_data(x_path1, x_path2, x_path3, x_path4, x_path5, y_path, random_seed=None):
    #all_curr=np.load(x_path,allow_pickle=True)
    #all_next=np.load(y_path,allow_pickle=True)
    all_curr1 = pickle.load(open(x_path1, 'rb'))
    all_curr2 = pickle.load(open(x_path2, 'rb'))
    all_curr3 = pickle.load(open(x_path3, 'rb'))
    all_curr4 = pickle.load(open(x_path4, 'rb'))
    all_curr5 = pickle.load(open(x_path5, 'rb'))

    all_next=np.load(y_path,allow_pickle=True)
    
    #x_array = all_curr
    x_array = np.concatenate((all_curr1, all_curr2, all_curr3, all_curr4, all_curr5), axis=0)
    y_array = all_next

    #x_array = np.vstack(all_curr)
    #y_array = np.vstack(all_next)

    if random_seed is None:
        rand_idx=np.arange(0, np.shape(x_array)[0])
    else:
        np.random.seed(random_seed)
        rand_idx=np.random.choice(range(np.shape(x_array)[0]), size=np.shape(x_array)[0],replace=False)
    
    split_train_idx, split_val_idx = rand_idx[np.shape(x_array)[0]//10:],rand_idx[:np.shape(x_array)[0]//10] 
    x_train, x_val=np.asarray(x_array)[split_train_idx], np.asarray(x_array)[split_val_idx]
    y_train, y_val=np.asarray(y_array)[split_train_idx],np.asarray(y_array)[split_val_idx]
    
    #if global_normalize:
    #    x_train=x_train/x_train.max()
    #    y_train=y_train/y_train.max()
    #    x_val=x_val/x_val.max()
    #    y_val=y_val/y_val.max()
    return x_train,y_train,x_val,y_val

In [12]:
segs_path1 = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/{}/segs_list_{}_32_part1.pkl'.format(bird, bird)     ###change
segs_path2 = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/{}/segs_list_{}_32_part2.pkl'.format(bird, bird)     ###change
segs_path3 = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/{}/segs_list_{}_32_part3.pkl'.format(bird, bird)     ###change
segs_path4 = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/{}/segs_list_{}_32_part4.pkl'.format(bird, bird)     ###change
segs_path5 = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/{}/segs_list_{}_32_part5.pkl'.format(bird, bird)     ###change
next_path = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/{}/next_list_{}_32.pkl'.format(bird, bird)            ###change

In [13]:
#x_train,y_train,x_val,y_val = extract_spec_data(segs_path, next_path, random_seed=0,global_normalize=True)
x_train1,y_train1,x_val1,y_val1 = extract_spec_data(segs_path1,segs_path2,segs_path3,segs_path4,segs_path5,next_path,random_seed=0)

In [14]:
x_train = x_train1[:, :32, :]
x_val = x_val1[:, :32, :]
y_train = y_train1[:, :32]
y_val = y_val1[:, :32]

In [15]:
np.shape(x_train), np.shape(x_val), np.shape(y_train)

((243675, 32, 32), (27074, 32, 32), (243675, 32))

### Training network

In [ ]:
train_dset = tf.data.Dataset.from_tensor_slices((x_train.astype(np.float32), y_train.astype(np.float32)))
train_dset = train_dset.shuffle(buffer_size=x_train.shape[0]+256).batch(64)
test_losses = []

optimizer = tf.optimizers.Adam(1e-3)
model = Temporal_Specgram_CNN_Model(optimizer=optimizer)
x_val, y_val = x_val.astype(np.float32), y_val.astype(np.float32)
with tf.device('/device:gpu:0'):
#     tf.print('Training Fold {}'.format(index))
    #model.load_weights('./temporal_specgram_weights/initial')
    for epoch in range(500):
        if epoch>250:
            model.optimizer.learning_rate=2e-4
        for step, train_batch in enumerate(train_dset):
            train_loss = model.train_model(train_batch[0], train_batch[1])

        tf.print('Epoch {}, Train-Loss {}'.format(epoch, train_loss), output_stream=sys.stdout)
        test_losses.append(model.compute_test_loss(x_val, y_val))
        
    plt.plot(test_losses)
    plt.xlabel('Epochs')
    plt.ylabel('Test Loss')
    plt.title('Test Loss by Epoch: {} units'.format(256))
    #plt.savefig('mnt/cube/srrudrar/temporal_model/loss_plots/{}units_loss.png'.format(256))

In [ ]:
with tf.device('/device:gpu:3'):
    #model.save_weights('/mnt/cube/srrudrar/temporal_model/temporal_model_weights/temp_256_l1_3_5_500epoch_64batch.h5')
    #predicted = model.full_model(x_val)
    enc_train1 = model.enc_2(model.enc_dropout(model.enc_1(x_train[:15000,:,:][:,:,:,np.newaxis])))
    enc_train2 = model.enc_2(model.enc_dropout(model.enc_1(x_train[15000:30000,:,:][:,:,:,np.newaxis])))
    enc_train3 = model.enc_2(model.enc_dropout(model.enc_1(x_train[30000:45000,:,:][:,:,:,np.newaxis])))
    enc_train4 = model.enc_2(model.enc_dropout(model.enc_1(x_train[45000:60000,:,:][:,:,:,np.newaxis])))
    enc_train5 = model.enc_2(model.enc_dropout(model.enc_1(x_train[60000:80000,:,:][:,:,:,np.newaxis])))
    enc_train6 = model.enc_2(model.enc_dropout(model.enc_1(x_train[80000:,:,:][:,:,:,np.newaxis])))
    enc_val = model.enc_2(model.enc_dropout(model.enc_1(x_val[:,:,:,np.newaxis])))
    #tf.print(predicted.shape)
    tf.print(enc_train1.shape)
    tf.print(enc_train2.shape)
    tf.print(enc_train3.shape)
    tf.print(enc_val.shape)

In [ ]:
enc_train = np.vstack([enc_train1, enc_train2, enc_train3, enc_train4, enc_train5, enc_train6])
enc_val = np.vstack([enc_val])

enc_train_reshape = np.reshape(enc_train, (len(enc_train),32,16))

In [ ]:
np.shape(enc_train), np.shape(x_train), np.shape(enc_val), np.shape(x_val)

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(4,5))
cax = ax.matshow(x_train[100], interpolation='nearest', aspect='auto', cmap=plt.cm.afmhot, origin='lower')
fig.colorbar(cax)
plt.title('Mel Spectrogram')

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(4,5))
cax = ax.matshow(enc_train_reshape[100], interpolation='nearest', aspect='auto', cmap=plt.cm.afmhot, origin='lower')
fig.colorbar(cax)
plt.title('Mel Spectrogram')

In [ ]:
resp_train = x_train1[:, 32:, :]
spec_train = [x.flatten() for x in x_train]
#enc_train

resp_test = x_val1[:, 32:, :]
spec_test = [x.flatten() for x in x_val]
#enc_test = enc_val
#x_val = x_val[:, :16, :]
#y_train = y_train[:, :16]
#y_val = y_val[:, :16]

In [ ]:
np.shape(spec_train), np.shape(enc_train), np.shape(resp_train), np.shape(spec_test), np.shape(enc_val), np.shape(resp_test)

In [ ]:
d = 512
bird = 'B1596'
np.save('/mnt/cube/srrudrar/temporal_cnn_model/{}/32x32/stim_resp_32x32_{}/spec_train_{}.npy'.format(bird,d,d), spec_train)
np.save('/mnt/cube/srrudrar/temporal_cnn_model/{}/32x32/stim_resp_32x32_{}/enc_train_{}.npy'.format(bird,d,d), enc_train)
np.save('/mnt/cube/srrudrar/temporal_cnn_model/{}/32x32/stim_resp_32x32_{}/resp_train_{}.npy'.format(bird,d,d), resp_train)

np.save('/mnt/cube/srrudrar/temporal_cnn_model/{}/32x32/stim_resp_32x32_{}/spec_test_{}.npy'.format(bird,d,d), spec_test)
np.save('/mnt/cube/srrudrar/temporal_cnn_model/{}/32x32/stim_resp_32x32_{}/enc_test_{}.npy'.format(bird,d,d), enc_val)
np.save('/mnt/cube/srrudrar/temporal_cnn_model/{}/32x32/stim_resp_32x32_{}/resp_test_{}.npy'.format(bird,d,d), resp_test)
        
np.save('/mnt/cube/srrudrar/temporal_cnn_model/{}/32x32/stim_resp_32x32_{}/y_train_{}.npy'.format(bird,d,d), y_train)
np.save('/mnt/cube/srrudrar/temporal_cnn_model/{}/32x32/stim_resp_32x32_{}/y_test_{}.npy'.format(bird,d,d), y_val)

In [ ]:
import json

In [ ]:
json.dump([float(i[0].numpy()) for i in test_losses],open('pred_only_64_channels_dropout_test_loss.json', 'w'))